In [58]:
import numpy as np
import tensorflow as tf
import csv
import pandas as pd
import keras
import numpy as np
from tensorflow.keras.layers import Flatten, Conv2D, Dense, MaxPooling2D
from art.attacks.evasion import FastGradientMethod, CarliniLInfMethod
from tensorflow.keras.datasets import mnist
from art.estimators.classification import KerasClassifier
from art.estimators.classification import TensorFlowV2Classifier
from tensorflow.keras.datasets import mnist

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Load the pre-trained model
model = tf.keras.models.load_model('mnist_CNN_model.h5')

In [54]:
# Define ART classifier
classifier = TensorFlowV2Classifier(model=model, nb_classes=10, input_shape=(28, 28, 1), loss_object=tf.keras.losses.CategoricalCrossentropy())

# Generate adversarial examples
epsilon = 0.1
fgsm = FastGradientMethod(classifier, eps=epsilon)
x_test_adv_fgsm = fgsm.generate(x_test)

# Make predictions
preds_true = np.argmax(y_test, axis=1)
preds_original = np.argmax(model.predict(x_test), axis=1)
preds_adv = np.argmax(model.predict(x_test_adv_fgsm), axis=1)

# Save output to CSV
output_data = []
for i in range(len(x_test)):
    output_data.append([i, epsilon, preds_true[i], preds_original[i], preds_adv[i]])

# Create DataFrame and save to CSV
df = pd.DataFrame(output_data, columns=['Image Index', 'Epsilon', 'Ground Truth', 'Prediction without Adv', 'Prediction with Adv'])
df.to_csv('adversarial_output.csv', index=False)


C:\Users\achme\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\achme\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [56]:
# create a data summary log

preds_original = np.argmax(model.predict(x_test, batch_size=256), axis=1)  # Original predictions
preds_adv = np.argmax(model.predict(x_test_adv_fgsm, batch_size=256), axis=1)  # Adversarial predictions
preds_true = np.argmax(y_test, axis=1)  # True labels

# Compute Metrics
successful_attacks = np.sum(preds_adv != preds_true)
success_rate = (successful_attacks / len(x_test)) * 100

# Vectorized confidence calculation 
conf_before = np.mean(np.max(tf.nn.softmax(model.predict(x_test)), axis=1))
conf_after = np.mean(np.max(tf.nn.softmax(model.predict(x_test_adv_fgsm)), axis=1))

# Save Results 
summary_data = {
    'Epsilon': epsilon,
    'Total Successful Attacks': successful_attacks,
    'Success Rate (%)': success_rate,
    'Average Confidence (Before Attack)': conf_before,
    'Average Confidence (After Attack)': conf_after,
}
pd.DataFrame([summary_data]).to_csv("Epsilon_Summary.csv", index=False)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
